# Trafikverket EDA

## Load the modules

Load the necessary modules and create a function to get traffic information from Trafikverket's API.

In [26]:
import requests
from dotenv import load_dotenv
import os
import pprint

load_dotenv()
API_key = os.getenv('API_KEY')

url = 'https://api.trafikinfo.trafikverket.se/v2/data.json'

headers = {
    'Content-Type': 'application/xml', 
}

filter = """<EQ name = "Deleted" value = "false"/>"""

# dictionary with object types and their corresponding schema versions
object_types = {
    # Väg - Trafikinformation
    'Camera': 1,
    'FerryAnnouncement': 1.2, 
    'FerryRoute': 1.2,
    'FrostDepthMeasurepoint': 1,
    'FrostDepthObservation': 1,
    'Icon': 1.1,
    'Parking': 1.4,
    'RoadCondition': 1.2,
    'Situation': 1.5, 
    'TrafficFlow': 1.5, 
    'TravelTimeRoute': 1.6,
    'WeatherMeasurepoint': 2.1,
    'WeatherObservation': 2.1,   
    # Järnväg - Trafikinformation
    'RailCrossing': 1.5, 
    'ReasonCode': 1, 
    'TrainAnnouncement': 1.9,
    'TrainMessage': 1.7,
    'TrainPosition': 1.1,
    'TrainStation': 1.5,
    'TrainStationMessage': 1}

def xml_query(API_key, filter, object_type, limit = None):
    if limit != None:
        query = f'''<QUERY objecttype="{object_type}" schemaversion="{object_types[object_type]}" limit="{limit}">'''

    else:
      query = f'''<QUERY objecttype="{object_type}" schemaversion="{object_types[object_type]}">'''

    if object_type in ['Situation', 'TrafficFlow', 'TrainMessage']: # object types that can filter on CountyNo
        filter += f'''<EQ name = "CountyNo" value = "1"/>''' # CountyNo for Stockholm
    
    xml_data = f'''<?xml version="1.0" encoding="UTF-8"?>
      <REQUEST>
        <LOGIN authenticationkey="{API_key}"/>
        {query}
          <FILTER>{filter}</FILTER>
        </QUERY>
      </REQUEST>'''
    
    return xml_data



In [27]:
xml_data = xml_query(API_key, filter, 'TrafficFlow', 10)

traffic_flow = requests.post(url=url, data=xml_data, headers=headers)

pprint.pprint(traffic_flow.text)

('{ '
 '"RESPONSE":{"RESULT":[{"TrafficFlow":[{"SiteId":40,"MeasurementTime":"2024-10-15T18:19:00.000+02:00","MeasurementOrCalculationPeriod":60,"VehicleType":"anyVehicle","VehicleFlowRate":840,"AverageVehicleSpeed":62.92,"CountyNo":1,"Deleted":false, '
 '"Geometry":{"SWEREF99TM":"POINT (677754.96 6578623.19)","WGS84":"POINT '
 '(18.122711 '
 '59.308964)"},"RegionId":4,"DataQuality":"bad","SpecificLane":"lane1","MeasurementSide":"unknown","ModifiedTime":"2024-10-15T16:19:05.020Z"},{"SiteId":4306,"MeasurementTime":"2024-10-15T18:19:00.000+02:00","MeasurementOrCalculationPeriod":60,"VehicleType":"anyVehicle","VehicleFlowRate":720,"AverageVehicleSpeed":67.8,"CountyNo":1,"Deleted":false, '
 '"Geometry":{"SWEREF99TM":"POINT (674788.02 6583302.84)","WGS84":"POINT '
 '(18.074469 '
 '59.352165)"},"RegionId":4,"DataQuality":"good","SpecificLane":"lane2","MeasurementSide":"unknown","ModifiedTime":"2024-10-15T16:19:12.836Z"},{"SiteId":861,"MeasurementTime":"2024-10-15T18:19:00.000+02:00","Measure